In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! unzip /content/gdrive/My\ Drive/anime\ face\ generation/faces.zip

In [1]:
import tensorflow as tf
import sys 
sys.path.append('/content/gdrive/My Drive/i2v')
import i2v #https://github.com/rezoo/illustration2vec
from PIL import Image
import numpy as np
!mkdir features


mkdir: cannot create directory ‘features’: File exists


In [2]:
illust2vec = i2v.make_i2v_with_chainer(
    "/content/gdrive/My Drive/i2v/illust2vec_ver200.caffemodel")

In [3]:
import os
def get_images(dir):
  folders = os.listdir(dir)
  images = {}
  for folder in folders: 
    images[folder]  = os.listdir(dir+ '/' +folder)
  return images

In [4]:
def path_to_name(path):
    name = path.split('/')
    name = name[-1]
    name = name.split('.')
    name = name[0]
    return name

In [5]:
save_dir = '/content/features'
processed = 0
def one_image(path , save_dir = save_dir , i2v = illust2vec ):
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.asarray(img)
    feature  = i2v.extract_feature([img])
    name = path_to_name(path)
    np.save('{}/{}_{}'.format(save_dir,name,'feature' ),feature)
    global processed 
    processed +=1 
    return

In [6]:
import random
def folder_to_paths(dir):
    dic = get_images(dir)
    paths = []
    for folder in dic:
        names = dic[folder]
        pths = ['{}/{}/{}'.format(dir,folder,name) for name in names]
        paths.extend(pths)
    random.shuffle(paths)
    return paths

In [7]:
failed = []
def paths_to_np(paths):
    global failed
    for path in paths:
      try:
        one_image(path) 
      except Exception as err: 
        print('Exception when processing {}'.format(name(path)))
        print(err.args)
        print(err)
        print(type(err))
        failed.append(path)
    return


In [10]:
import time
import threading
def multiprocessing(dir, num_threads):
  paths = folder_to_paths(dir)
  total = int(len(paths))
  print('{} pictures will be processed'.format(total))
  path_per_process = int(len(paths)/num_threads)
  threads = []
  starting = time.time()
  for i in range(int(num_threads +1 )):
    thread = threading.Thread(target = paths_to_np , args = ([paths[i*path_per_process:(i+1)*path_per_process]]))
    thread.start()
    threads.append(thread)

  while True: 
    time.sleep(5)
    sequence_finished = True
    for thread in threads: 
      sequence_finished &=  not thread.is_alive()
    if(sequence_finished):
      print('processing completed {}: processed'.format(processed))
      break
    else: 
      print('{} processed {} remaining {}% completed'.format(processed, total - processed, processed / total * 100))

  finishing = time.time()
  print('finished in {} seconds'.format(finishing - starting))
  return

In [ ]:
processed = 0
dir = '/content/faces'
multiprocessing(dir, 100)

In [12]:
!zip -r /content/getchu_features.zip /content/features

Streaming output truncated to the last 5000 lines.
  adding: content/features/819813char0_face0_feature.npy (deflated 7%)
  adding: content/features/660711char2_face0_feature.npy (deflated 7%)
  adding: content/features/880807char1_face0_feature.npy (deflated 7%)
  adding: content/features/431285char15_face0_feature.npy (deflated 8%)
  adding: content/features/795844char2_face0_feature.npy (deflated 8%)
  adding: content/features/26747char8_face0_feature.npy (deflated 7%)
  adding: content/features/660658char19_face0_feature.npy (deflated 7%)
  adding: content/features/792202char21_face0_feature.npy (deflated 7%)
  adding: content/features/270825char0_face0_feature.npy (deflated 7%)
  adding: content/features/653145char4_face0_feature.npy (deflated 8%)
  adding: content/features/679907char0_face0_feature.npy (deflated 8%)
  adding: content/features/262965char1_face0_feature.npy (deflated 8%)
  adding: content/features/212741char0_face0_feature.npy (deflated 8%)
  adding: content/featur